In [34]:
import os.path
import pandas as pd
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import datetime as dt
from google.api_core import exceptions
import matplotlib.pyplot as plt

In [35]:
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']
today = dt.date.today().strftime("%m_%d_%Y")
# this is ID for recon live: instilend
SPREADSHEET_ID = '1pzlQEQE4FI4OnGrhKMfu1Skwrvs-FubYuEn-Ez5HpsM'
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
'''flow = InstalledAppFlow.from_client_secrets_file('client_secret_510599516312-aj6d72c90n3fmrbf3ou6gromil06pr8c.apps.googleusercontent.com.json', SCOPES)
creds = flow.run_local_server(port=0)
with open('token.json', 'w') as token:
    token.write(creds.to_json())'''
service = build('drive', 'v3', credentials=creds)

# Call the Drive v3 API
results = service.files().export_media(
        fileId = SPREADSHEET_ID,
        mimeType = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet').execute()#
path = f"source/Portolio_History_{today}.xlsx"
with open(path, "wb") as f:
    f.write(results)
    f.close()

In [36]:
pivot_data = pd.read_excel(path, 'Pivot_Data')

In [72]:
total_coins = pivot_data.groupby(['Timestamp - UTC','Coin'])['# of Coins'].sum().reset_index()
total_coins.columns = total_coins.columns[:-1].tolist() + ['Total Coins']
pivot_data2 = pivot_data.merge(total_coins, on=['Timestamp - UTC','Coin'], how="left")
pivot_data2['% of Coins'] = pivot_data2['# of Coins']/pivot_data2['Total Coins']
pivot_data2['Weighted_APY'] = pivot_data2['APY'] * pivot_data2['% of Coins']
pivot_data2['Weighted_COFA'] = pivot_data2['COFA'] * pivot_data2['% of Coins']
apy_history = pivot_data2.groupby(['Timestamp - UTC','Coin'])[['Weighted_APY','Weighted_COFA']].sum().reset_index()

In [ ]:
apy_history['Timestamp - UTC'] = pd.to_datetime(apy_history['Timestamp - UTC'])
for coin in apy_history['Coin'].unique():
    plt.figure(figsize=(10,5))
    plt.scatter(x='Timestamp - UTC', y='Weighted_APY', label='APY', color='b', data=apy_history[apy_history['Coin']==coin])
    plt.scatter(x='Timestamp - UTC', y='Weighted_COFA', label='COFA', color='r', data=apy_history[apy_history['Coin']==coin])
    #apy_history[apy_history['Coin']==coin].plot.scatter(x='Timestamp - UTC', y='Weighted_APY', label='APY', color='blue')
    #apy_history[apy_history['Coin']==coin].plot.scatter(x='Timestamp - UTC', y='Weighted_APY', label='COFA', color='red')
    plt.title(f"APY-COFA History - {coin}")
    plt.xticks(rotation=45)
    plt.legend()
    plt.plot()